In [267]:
import pandas as pd
import numpy as np
import re
import warnings

warnings.filterwarnings("ignore")

# <font color="#666666">Functions</font>

In [268]:
def leisure_pretreat(dframe, turnOnAvg=0, choice_yr=None, input_cat=None) : 

    """
    
    Ver. 7
    
    여가 관련 통계조사 전처리 함수입니다.

    -------------------------------------------------------------------------------

    1. (Ver. 2 -> Ver. 3 : '평균' 컬럼을 선택적으로 제외할 수 있게 수정)

    'turnOnAvg' 파라미터에 1을 입력하면 '평균' 컬럼을 유지합니다.

    2. (Ver. 3 -> Ver. 4 : '연도'를 입력하면 해당 연도 이상의 연도 리스트만 반환)

    'choice_yr' 파라미터에 연도를 입력하면, 해당 연도 이상의 컬럼만 반환합니다.

    3. (Ver. 4 -> Ver. 5 : 행 인덱스를 reset_index)

    4. (Ver. 5 -> Ver. 6 : 카테고리와 세부 카테고리로 나눔)

    5. (Ver. 6 -> Ver. 7 : 여가 종류를 하나의 카테고리로 묶음)

    -------------------------------------------------------------------------------

    """

    year_List = list(range(2016,2022))
    year_List = [value for value in year_List if value != 2017]   # 2017년도 빼기

    if choice_yr : 
        year_List = [value for value in year_List if value >= choice_yr]

    category = ["연령별", "가구소득별"]

    leis_sep = dframe.iloc[:,:2]

    conc_list_age = []
    conc_list_inc = []

    for yr in year_List : 

        pattern = str(yr)
        df_name = "DataFrame_" + str(yr)

        globals()[df_name] = dframe.filter(like=pattern)    # 조건에 만족하는 데이터 프레임 반환
        globals()[df_name] = pd.concat([leis_sep, globals()[df_name]], axis=1)     # 왼쪽 오른쪽으로 합치기, 인덱스가 다르면 outer join
        globals()[df_name] = globals()[df_name].replace("-",0)
        globals()[df_name].columns = globals()[df_name].iloc[0,:].to_list()
        globals()[df_name] = globals()[df_name].iloc[1:,:]
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(float)

        if turnOnAvg == 1:
            col_avg = globals()[df_name]["평균"]

        if "평균" in globals()[df_name].columns.to_list() : 
            globals()[df_name].drop("평균", axis=1, inplace=True)
            

        globals()[df_name].iloc[:,3:] = globals()[df_name].iloc[:,3:].apply(lambda x : x * globals()[df_name].iloc[:,2] / 100)  # 백분율로 표시된 값을 실제 값으로 변환
        globals()[df_name] = globals()[df_name].drop("표본수",axis=1)  # 합계 부분 삭제
        globals()[df_name].iloc[:,2:] = globals()[df_name].iloc[:,2:].astype(int)

        # 평균 컬럼을 보기 원한다면
        if turnOnAvg == 1 :
            globals()[df_name] = pd.concat([globals()[df_name], col_avg], axis=1)
            
        for cat1 in category : 

            df_name_c1 = "DataFrame_" + str(yr) + cat1
            globals()[df_name_c1] = globals()[df_name][globals()[df_name]["통계분류(1)"]==cat1].iloc[:,1:]
            globals()[df_name_c1].insert(0, "연도별", yr)
            globals()[df_name_c1].rename({"통계분류(2)":"세부 조사대상"},axis=1,inplace=True)
            globals()[df_name_c1].insert(1, "조사대상", cat1)

            if input_cat : 
                globals()[df_name_c1].insert(1, "여가 종류", input_cat)

            if cat1 == category[0] : 
                conc_list_age.append(globals()[df_name_c1])     # 연령별과 같으면
            else : 
                conc_list_inc.append(globals()[df_name_c1])     # 수입과 같으면

    df1 = pd.concat(conc_list_age, axis=0)
    df2 = pd.concat(conc_list_inc, axis=0)

    df1.fillna(0, axis=1, inplace=True)
    df2.fillna(0, axis=1, inplace=True)

    # 연령과 수입을 join
    df_conc = pd.concat([df1, df2], axis=0)
    df_conc.reset_index(drop=True, inplace=True)   #원래 있던 인덱스를 컬럼으로 만들지 않고 버림

    df_conc = df_conc.rename(columns=lambda x: x.replace(" ",""))
    
    return df_conc

In [269]:
def csv_to_excel(dataframe, filename) : 

    import os

    dir_path = "./pretreat_data/"
    xlsx_dir = os.path.join(dir_path,str(filename))

    dataframe.to_excel(excel_writer=xlsx_dir)

# 여가 관련 자료

## <font color=#FFB6C1> 세계 행복 지수 </font>

In [270]:
happy_world = pd.read_excel("G:/내 드라이브/취미/세계행복지수_2020~2022.xlsx")
happy_world.head()

,순위,국가,행복지수
0,1,핀란드,7.804
1,2,덴마크,7.586
2,3,아이슬란드,7.530
3,4,이스라엘,7.473
4,5,네덜란드,7.403


In [271]:
happy_world

,순위,국가,행복지수
0,1,핀란드,7.804
1,2,덴마크,7.586
2,3,아이슬란드,7.530
3,4,이스라엘,7.473
4,5,네덜란드,7.403
...,...,...,...
132,133,콩고 민주공화국,3.207
133,134,짐바브웨,3.204
134,135,시에라리온,3.138
135,136,레바논,2.392


## <font color=#E19B50> OECD 국가별 시간 사용 </font>

In [272]:
"""
1. Paid work or study : 근무 시간, 대학교 등을 포함한 정규 교육 시간(숙제 포함), 구직 활동 등을 포함한 시간

2. unpaid work : 집안일, 쇼핑, 양육, 가족 여행 등 가족 구성원과 관련된 시간

3. personal care : 잠, 식사, 휴식 등 건강을 유지하기 위해 필요한 시간

4. leisure : 문화 생활, 스포츠 / 게임, 야외 활동 등 즐거움을 위한 사회 / 개인 활동

5. others : 종교 혹은 그 이외의 활동


시간은 분(min) 단위임
"""

timeUse_oecd = pd.read_csv("G:/내 드라이브/취미/TIME_USE_25052023035411993.csv", encoding="cp949",sep="\t")
# timeUse_oecd.info()
# timeUse_oecd.head()

In [273]:
# 성별에 Total을 없앰
timeUse_oecd = timeUse_oecd[timeUse_oecd["SEX"] != "TOTAL"]

In [274]:
# value 컬럼명을 minutes로 변경
timeUse_oecd.rename({"Value":"minutes"}, axis=1, inplace=True)

In [275]:
# 아래와 같은 순서로 오름차순 후 인덱스 리셋
timeUse_oecd = timeUse_oecd.sort_values(by=["Country", "SEX", "Description"], ascending=True)
timeUse_oecd = timeUse_oecd.reset_index(drop=True)
timeUse_oecd.columns = ["국가", "분류", "성별", "분(min)"]
timeUse_oecd.head()

,국가,분류,성별,분(min)
0,Australia,Leisure,MEN,297.031370
1,Australia,Other,MEN,18.062717
2,Australia,Paid work or study,MEN,304.055760
3,Australia,Personal care,MEN,649.254330
4,Australia,Unpaid work,MEN,171.595830


In [276]:
# csv_to_excel(timeUse_oecd, "OECD 국가별 시간 사용.xlsx")

## <font color=#FFE0C6> 행복수준 </font>

In [277]:
lev_happiness = pd.read_csv("G:/내 드라이브/취미/행복수준_20230510094831.csv", encoding="cp949")

In [278]:
lev_happiness = leisure_pretreat(lev_happiness, turnOnAvg=1)
lev_happiness.tail()

,연도별,조사대상,세부조사대상,1-2점,3-4점,5-6점,7점이상,평균
65,2021,가구소득별,200-300만원 미만,11,58,504,846,6.8
66,2021,가구소득별,300-400만원 미만,5,87,551,1251,7.0
67,2021,가구소득별,400-500만원 미만,4,41,449,1113,7.1
68,2021,가구소득별,500-600만원,4,26,354,996,7.2
69,2021,가구소득별,600만원 이상,6,25,342,1354,7.4



## <font color="#FF5675"> 일과 여가 생활 균형 </font>

In [279]:
leisure_bal = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')

In [280]:
leisure_bal = leisure_pretreat(leisure_bal)
leisure_bal.tail()

,연도별,조사대상,세부조사대상,일에더집중하고있다,일에조금더집중하고있다,다소일에집중하고있다,균형을이루고있다,다소여가에더집중하고있다,여가에조금더집중하고있다,여가에더집중하고있다
65,2021,가구소득별,200~300만원,53,204,285,599,154,107,12
66,2021,가구소득별,300~400만원,45,267,401,802,227,134,15
67,2021,가구소득별,400~500만원,38,211,317,708,231,96,6
68,2021,가구소득별,500~600만원,22,178,255,630,197,92,6
69,2021,가구소득별,600만원 이상,41,231,361,735,247,103,6


## <font color="#FFD73C"> 평일 여가 활동_관광 활동 </font>

In [281]:
trip = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_관광활동_20230510095154.csv", encoding="cp949")
# trip.info()
# trip.head()

In [282]:
trip = leisure_pretreat(trip, input_cat="관광")
trip.tail()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
65,2021,관광,가구소득별,200~300만원,0,0,1,51,162,223,87
66,2021,관광,가구소득별,300~400만원,0,0,3,59,207,304,130
67,2021,관광,가구소득별,400~500만원,0,0,0,58,217,273,109
68,2021,관광,가구소득별,500~600만원,0,0,0,57,191,236,94
69,2021,관광,가구소득별,600만원 이상,0,0,0,57,216,348,199


## <font color="#FFD73C"> 평일 여가 활동_문화예술 관람 활동 </font>

In [283]:
artCult = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_문화예술_관람활동_20230519170721.csv", encoding="cp949")
artCult.head()

,통계분류(1),통계분류(2),2016,2016.1,2016.2,2016.3,2016.4,2016.5,2016.6,2016.7,...,2020.6,2020.7,2021,2021.1,2021.2,2021.3,2021.4,2021.5,2021.6,2021.7
0,통계분류(1),통계분류(2),표본수,매우 불만족한다,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,...,매우 만족한다,평균,표본수,불만족한다,다소 불만족한다,보통이다,다소 만족한다,만족한다,매우 만족한다,평균
1,전체,소계,3957,0,0.1,0.3,5.2,28.5,55.5,10.4,...,13.7,5.7,2068,0,0.2,5.6,30.8,47.4,15.9,5.7
2,성별,남성,1687,0,0.2,0.3,4.9,30.6,54.2,9.7,...,13.8,5.7,1011,-,0.4,6.1,30.1,46.5,16.9,5.7
3,성별,여성,2270,0,0.1,0.2,5.4,26.7,56.6,11,...,13.7,5.7,1057,0,-,5.1,31.5,48.3,15,5.7
4,연령별,15~19세,388,0,0,-,7,29.7,54,9.4,...,18.8,5.8,175,-,-,2.5,39.1,32.2,26.2,5.8


In [284]:
artCult = leisure_pretreat(artCult, input_cat="문화 예술 관람")
artCult.head()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
0,2016,문화 예술 관람,연령별,15~19세,0.0,0.0,0,27,115,209,36
1,2016,문화 예술 관람,연령별,20대,0.0,3.0,1,27,260,543,93
2,2016,문화 예술 관람,연령별,30대,0.0,0.0,2,42,231,449,83
3,2016,문화 예술 관람,연령별,40대,0.0,1.0,3,46,249,419,79
4,2016,문화 예술 관람,연령별,50대,0.0,0.0,1,26,156,315,63


## <font color="#FFD73C"> 평일 여가 활동_사회 및 기타 </font>

In [285]:
sc_etc = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_사회_및_기타활동_20230510093021.csv", encoding="cp949")

In [286]:
sc_etc = leisure_pretreat(sc_etc, input_cat="사회 및 기타")
sc_etc.tail()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
65,2021,사회 및 기타,가구소득별,200~300만원,0.0,0,7,157,390,573,193
66,2021,사회 및 기타,가구소득별,300~400만원,0.0,0,6,195,488,749,284
67,2021,사회 및 기타,가구소득별,400~500만원,1.0,0,2,137,457,667,199
68,2021,사회 및 기타,가구소득별,500~600만원,0.0,2,5,109,383,558,216
69,2021,사회 및 기타,가구소득별,600만원 이상,0.0,0,1,114,518,659,293


## <font color="#FFD73C"> 평일 여가 활동_스포츠 참여 활동 </font>

In [287]:
doSports = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_스포츠_참여활동_20230510095111.csv", encoding="cp949")

In [288]:
doSports = leisure_pretreat(doSports, input_cat="스포츠 참여")
doSports.tail()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
65,2021,스포츠 참여,가구소득별,200~300만원,0.0,0,2,44,166,253,121
66,2021,스포츠 참여,가구소득별,300~400만원,0.0,0,0,63,209,335,159
67,2021,스포츠 참여,가구소득별,400~500만원,0.0,0,0,60,217,300,141
68,2021,스포츠 참여,가구소득별,500~600만원,1.0,0,2,49,181,317,130
69,2021,스포츠 참여,가구소득별,600만원 이상,0.0,0,2,40,220,376,254


## <font color="#FFD73C"> 평일 여가 활동_스포츠 관람 활동 </font>

In [289]:
wacthSports = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_스포츠_관람활동_20230519170812.csv", encoding="cp949")

In [290]:
wacthSports = leisure_pretreat(wacthSports, input_cat="스포츠 관람")
wacthSports.head()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
0,2016,스포츠 관람,연령별,15~19세,0.0,0,0,46,147,150,40
1,2016,스포츠 관람,연령별,20대,0.0,0,2,69,273,388,73
2,2016,스포츠 관람,연령별,30대,1.0,0,10,125,337,384,65
3,2016,스포츠 관람,연령별,40대,3.0,0,5,133,388,429,61
4,2016,스포츠 관람,연령별,50대,2.0,0,0,81,328,424,50


## <font color="#FFD73C"> 평일 여가 활동_취미 오락 활동 </font>

In [291]:
games = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_취미・오락활동_20230510092915.csv", encoding="cp949")

In [292]:
games = leisure_pretreat(games, input_cat="취미 오락")
games.tail()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
65,2021,취미 오락,가구소득별,200~300만원,0,0,2,140,398,573,200
66,2021,취미 오락,가구소득별,300~400만원,0,0,5,164,512,819,252
67,2021,취미 오락,가구소득별,400~500만원,0,0,4,112,482,696,205
68,2021,취미 오락,가구소득별,500~600만원,0,1,1,119,395,544,233
69,2021,취미 오락,가구소득별,600만원 이상,0,0,0,137,499,677,338


## <font color="#FFD73C"> 평일 여가 활동_휴식 활동 </font>

In [293]:
takeArest = pd.read_csv("G:/내 드라이브/취미/평일에_참여한_여가활동_만족도_휴식활동_20230510092941.csv", encoding="cp949")

In [294]:
takeArest = leisure_pretreat(takeArest, input_cat="휴식")
takeArest.tail()

,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
65,2021,휴식,가구소득별,200~300만원,0,0,4,162,453,590,198
66,2021,휴식,가구소득별,300~400만원,0,1,5,215,564,837,233
67,2021,휴식,가구소득별,400~500만원,0,0,1,159,535,690,198
68,2021,휴식,가구소득별,500~600만원,0,0,4,133,440,594,194
69,2021,휴식,가구소득별,600만원 이상,0,1,1,145,551,707,302


## 평일 여가 활동 합치기

In [310]:
allLeisure = pd.concat([trip, artCult, sc_etc, doSports, wacthSports, games, takeArest], axis=0)
allLeisure.reset_index(drop=True, inplace=True)
print("전체 행 길이 :", len(allLeisure[:]))
allLeisure.head()

전체 행 길이 : 490


,연도별,여가종류,조사대상,세부조사대상,매우불만족한다,불만족한다,다소불만족한다,보통이다,다소만족한다,만족한다,매우만족한다
0,2016,관광,연령별,15~19세,0,0.0,0,34,112,204,34
1,2016,관광,연령별,20대,0,0.0,1,39,218,408,85
2,2016,관광,연령별,30대,0,1.0,3,40,275,414,116
3,2016,관광,연령별,40대,0,1.0,1,54,347,516,113
4,2016,관광,연령별,50대,0,0.0,0,35,317,505,120


In [311]:
allLeisure = allLeisure.melt(id_vars=['연도별', '여가종류', "조사대상", "세부조사대상"], \
                             value_vars=["매우불만족한다", "불만족한다", "다소불만족한다", "보통이다", "다소만족한다", "만족한다", "매우만족한다"], \
                            var_name='평가항목', value_name='응답인원')
print("전체 행 길이 :", len(allLeisure[:]))
allLeisure.head()

전체 행 길이 : 3430


,연도별,여가종류,조사대상,세부조사대상,평가항목,응답인원
0,2016,관광,연령별,15~19세,매우불만족한다,0
1,2016,관광,연령별,20대,매우불만족한다,0
2,2016,관광,연령별,30대,매우불만족한다,0
3,2016,관광,연령별,40대,매우불만족한다,0
4,2016,관광,연령별,50대,매우불만족한다,0


In [297]:
allLeisure.tail()

,연도별,여가종류,조사대상,세부조사대상,평가항목,응답인원
3425,2021,휴식,가구소득별,200~300만원,매우만족한다,198
3426,2021,휴식,가구소득별,300~400만원,매우만족한다,233
3427,2021,휴식,가구소득별,400~500만원,매우만족한다,198
3428,2021,휴식,가구소득별,500~600만원,매우만족한다,194
3429,2021,휴식,가구소득별,600만원 이상,매우만족한다,302


## <font color="#C1FF6B	"> 지난 1년 동안 지속적 반복적 참여 </font>

In [298]:
rep_per_year = pd.read_csv("G:/내 드라이브/취미/_지난_1년_동안__지속적_반복적_참여_여가활동_여부_20230510095327.csv", encoding="cp949")

In [299]:
rep_per_year = leisure_pretreat(rep_per_year)

## <font color="#00D7FF	"> 보다 나은 여가 생활 </font>

In [300]:
btr_leisure = pd.read_csv("G:/내 드라이브/취미/정책만족도_보다_나은_여가생활을_위해_관련_법규와_제도를_개선_20230510094233.csv", encoding="cp949")

In [301]:
btr_leisure = leisure_pretreat(btr_leisure, turnOnAvg=1)

## <font color="#90AFFF"> 근로시간 단축 이후 </font>

In [302]:
cut_worktime = pd.read_csv("G:/내 드라이브/취미/근로시간_단축_이후_여가만족도_20230510094931.csv", encoding="cp949")

In [303]:
cut_worktime = leisure_pretreat(cut_worktime, turnOnAvg=1, choice_yr=2019)

## <font color="#68269A"> 여가 생활 불만족 </font>

In [304]:
unsat_leis = pd.read_csv("G:/내 드라이브/취미/여가생활에_만족하지_못하는_이유_20230525113739.csv", encoding="cp949")

In [305]:
unsat_leis = leisure_pretreat(unsat_leis)

## <font color="#82B3ED"> OECD 평균 근로시간 </font>

In [306]:
avg_workt_oecd = pd.read_csv("G:/내 드라이브/취미/ANHRS_25052023055022396.csv", sep="\t", encoding="cp949")

In [307]:
avg_workt_oecd.rename({"Value":"hours"}, axis=1, inplace=True)

In [308]:
avg_workt_oecd = avg_workt_oecd[avg_workt_oecd["Employment status"] == "Total employment"]

In [309]:
avg_workt_oecd.reset_index(drop=True, inplace=True)
avg_workt_oecd.columns = ["국가", "전체 고용", "연도", "시간(hr)"]
avg_workt_oecd.head()

,국가,전체 고용,연도,시간(hr)
0,Australia,Total employment,2010,1777.570638
1,Australia,Total employment,2011,1773.895120
2,Australia,Total employment,2012,1770.775049
3,Australia,Total employment,2013,1766.009185
4,Australia,Total employment,2014,1755.132749


## 변수 리스트

In [ ]:
# var_list = ["happy_world", "timeUse_oecd", "lev_happiness", "leisure_bal_age", "leisure_bal_inc", "", "", "", "", "", "", "", "", "", "", "", "", "", \
#             "", ""]

# 일과 여가 생활 균형_연습버전

In [ ]:
leisure = pd.read_csv("G:/내 드라이브/취미/일과_여가생활_간_균형_20230510094439.csv", encoding='cp949')
# leisure.info()
# leisure.head(10)

In [ ]:
leisure_T = leisure.T   # 행열 전환
# leisure_T.head(5)

In [ ]:
leisure_T.replace("-",0,inplace=True)

In [ ]:
leis_uq = leisure.iloc[:,0].unique()
leis_uq = leis_uq.tolist()   # ndarray -> list
leis_uq = [re.sub(r"\(+.\)","",uq.replace(" ","_")) for uq in leis_uq]  # 괄호와 괄호 안 글자, 공백 제거
print(leis_uq)

In [ ]:
leisure_T.columns = leisure_T.iloc[0,:]
# leisure_T.info()

In [ ]:
columns_nospace = [re.sub(r"\(+.\)", "", col.replace(" ","_")) for col in leisure_T.columns]

In [ ]:
leisure_T.columns = columns_nospace
print(leisure_T.columns)

In [ ]:
# 통계 분류별로 데이터 프레임 분류

for cat in leis_uq : 
    globals()["leisure_{}".format(cat)] = leisure_T[cat]
    
    if type(globals()["leisure_{}".format(cat)]) == pd.core.series.Series :   # type이 시리즈라면
        globals()["leisure_{}".format(cat)] = pd.DataFrame(globals()["leisure_{}".format(cat)])   # 시리즈를 데이터프레임으로 변경
        
    globals()["leisure_{}".format(cat)].columns = globals()["leisure_{}".format(cat)].iloc[1,:]
    globals()["leisure_{}".format(cat)] = globals()["leisure_{}".format(cat)].iloc[2:,:]
    globals()["leisure_{}".format(cat)].iloc[:,1:] = globals()["leisure_{}".format(cat)].iloc[:,1:].astype(float)

In [ ]:
leis_uq2 = leis_uq[1:]
print(leis_uq2)

In [ ]:
for uq2 in leis_uq2 : 
    globals()["leis_{0}_{1}".format(uq2, "합본")] = pd.concat([leisure_통계분류, globals()["leisure_{}".format(uq2)]],axis=1)

In [ ]:
for uniq in leis_uq2 : 
    idx_List = globals()["leis_{0}_{1}".format(uniq, "합본")].index.to_list()   # 분리된 데이터프레임 인덱스 리스트로 반환
    
    for yr in year_List : 
        sep_year = []    # 정규표현식에 만족하는 인덱스를 저장할 리스트
        
        for idx in idx_List :
            find_yr = re.findall(str(yr)+"[.0-9]*",idx)     # 연도 + '.' or 숫자 1개 이상 포함돼 있는 인덱스 저장 

            if find_yr:      # 빈 리스트가 아니라면 (findall은 조건에 안 맞으면 빈 리스트 반환)
                sep_year.append(find_yr[0])

        Last_idx = len(sep_year)
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, "합본")][:][sep_year[0]:sep_year[Last_idx-1]]    # 행 인덱스로 슬라이싱, 행 인덱스로 슬라이싱할 때 콜론 뒤 값을 포함해서 슬라이싱한다.
        pattern = str(yr) + r'[.0-9]*'
        new_index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.map(lambda x: re.sub(pattern, str(yr), x))   # map 함수를 이용해 인덱스마다 정규 표현식 sub를 적용
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = new_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].index = globals()["leis_{0}_{1}".format(uniq, str(yr))].index.astype(int)   # 인덱스를 int 타입으로
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"통계분류(2)" : "분류"}, inplace=True)   # 컬럼명 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].reset_index()   # reset_index
        globals()["leis_{0}_{1}".format(uniq, str(yr))].rename(columns={"index" : "연도"}, inplace=True)    # 인덱스가 컬럼으로 바뀌고, 그 컬럼명을 변경
        globals()["leis_{0}_{1}".format(uniq, str(yr))] = globals()["leis_{0}_{1}".format(uniq, str(yr))].set_index(["연도","분류"])   # 멀티 인덱스 설정


# 전체 -> 리스트 중에서 2016년이면 리스트에 저장 -> 리스트에 저장된 첫번째와 마지막 인덱스로 슬라이싱 -> 새로운 데이터 프레임 생성

In [ ]:
leis_17개_시도별_2016_T = leis_17개_시도별_2016.T
leis_17개_시도별_2016_T.head(10)

In [ ]:
df = leis_17개_시도별_2016_T.stack(level=0)
pd.DataFrame(df)


In [ ]:
pd.concat([leis_17개_시도별_2016, leis_가구소득별_2016],axis=1)

## PDF 불러오기

In [ ]:
import fitz
path1 = "G:/내 드라이브/취미/세계행복보고서_2023.pdf"

In [ ]:
doc = fitz.open(path1)

pages = list(range(36,39))

count = 0

for page in doc : 
    
    count += 1

    if count in pages : 
        globals()['text_wrd_hap'+str(count)] = page.get_text()
        globals()['text_wrd_hap'+str(count)] = globals()['text_wrd_hap'+str(count)].split("\n")

doc.close()

In [ ]:
for pg in pages : 
    length = len(globals()['text_wrd_hap'+str(pg)])
    print("리스트의 길이 :", length)
    print("-*-*-*-*-*-*-*-*-*-*-*-*-*-")
    print("리스트 앞에 4개의 원소 :", globals()['text_wrd_hap'+str(pg)][:10])
    print("-*-*-*-*-*-*-*-*-*-*-*-*-*-")
    print("끝에서 3개의 원소 :", globals()['text_wrd_hap'+str(pg)][-10:])
    print("\n\n")